In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler


df = pd.read_csv("/Users/sangrimkoo/Desktop/프로젝트/사용하는데이터/VIP데이터.csv")

In [3]:
df = df.sort_values(by=['발급회원번호', '기준년월'])

In [4]:
def calc_slope_6m(series):
    # 데이터가 2개 미만이거나 전부 0이면 계산 불가 (0 반환)
    if len(series) < 2 or np.sum(series) == 0:
        return 0
    
    y = series.values
    x = np.arange(len(y))
    
    # 값이 6개월 내내 똑같으면 기울기 0
    if np.all(y == y[0]): return 0
    
    # 선형 회귀로 기울기 계산
    slope, _, _, _, _ = linregress(x, y)
    
    # 계산 오류(NaN)가 나면 0으로 처리
    if np.isnan(slope): return 0
    
    return slope

In [5]:
print("고객별 추세(기울기) 계산 중... (시간이 좀 걸릴 수 있어!)")

# 각 지표별로 6개월 기울기 계산
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)

# 잔액 컬럼 이름 확인 필요 (보통 '잔액_합계_B0M' 또는 '잔액_B0M')
balance_col = '잔액_B0M' if '잔액_B0M' in df.columns else df.columns[df.columns.str.contains('잔액')][0]
df['Slope_Balance'] = df.groupby('발급회원번호')[balance_col].transform(calc_slope_6m)

df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# 가장 최근 월(12월) 데이터만 추출 (현재 시점 기준 분석)
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

print(f"분석 대상 월: {target_month}")

고객별 추세(기울기) 계산 중... (시간이 좀 걸릴 수 있어!)
분석 대상 월: 201812


In [6]:
scaler = MinMaxScaler()

def convert_slope_to_score(df_in, col_name):
    # 감소(음수)하는 경우만 추출하여 양수로 변환 (감소폭이 클수록 높은 점수)
    # 증가하거나 변동 없는 경우(0 이상)는 0점으로 처리 (위험하지 않으니까!)
    negative_slopes = df_in[col_name].apply(lambda x: -x if x < 0 else 0)
    
    # 0~1 사이로 점수 압축 (MinMax Scaling)
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

# 각 지표별 점수 계산
score_spend = convert_slope_to_score(df_final, 'Slope_Spend')
score_balance = convert_slope_to_score(df_final, 'Slope_Balance')
score_count = convert_slope_to_score(df_final, 'Slope_Count')

In [7]:
# 가중치 설정 (합계 100)
W_SPEND = 40   # 소비 감소 비중 (제일 중요!)
W_BALANCE = 30 # 잔액 감소 비중
W_COUNT = 20   # 빈도 감소 비중
W_RISK = 10    # 리스크 비중

# 리스크 점수 (연체나 거절 이력 있으면 1, 없으면 0)
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | 
    (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# 최종 점수 계산
df_final['Churn_Score'] = (
    (score_spend * W_SPEND) + 
    (score_balance * W_BALANCE) + 
    (score_count * W_COUNT) + 
    (has_risk * W_RISK)
)

# 보기 좋게 소수점 첫째 자리까지만 남김
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

In [8]:
print(f"\n[Churn_Score 생성 완료]")
print(f"전체 고객 수: {len(df_final)}명")
print(f"점수 평균: {df_final['Churn_Score'].mean():.2f}점")
print(f"점수 최대값: {df_final['Churn_Score'].max()}점")

print("\n[상위 5명 데이터 확인]")
cols_view = ['발급회원번호', 'Slope_Spend', 'Slope_Balance', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head())


[Churn_Score 생성 완료]
전체 고객 수: 1910명
점수 평균: 8.97점
점수 최대값: 63.7점

[상위 5명 데이터 확인]
         발급회원번호    Slope_Spend  Slope_Balance  Churn_Score
10325  SYN_3968 -243906.428571 -117226.657143         63.7
10596  SYN_5296 -206826.314286 -381888.542857         55.4
11267  SYN_9037 -229686.857143 -119831.371429         54.9
10230  SYN_3442 -168908.285714 -186469.485714         51.4
9864   SYN_1527 -186951.657143 -241112.200000         49.6


In [9]:
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

# 1. 상위 2000명만 샘플링
df_modeling = df_final.head(2000).copy()

# 2. 학습에 사용할 Feature와 Target 설정
# Target: 우리가 만든 'Churn_Score'
# Feature: 숫자형 데이터 전부 (단, 식별자나 Target 자체는 제외)
target_col = 'Churn_Score'
ignore_cols = ['발급회원번호', '기준년월', target_col]

# 숫자형 컬럼만 선택
feature_cols = df_modeling.select_dtypes(include=['number']).columns
feature_cols = [c for c in feature_cols if c not in ignore_cols]

X = df_modeling[feature_cols]
y = df_modeling[target_col]

print(f"모델링 대상 데이터: {len(X)}명")
print(f"사용할 변수(Feature) 개수: {len(feature_cols)}개")

/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


모델링 대상 데이터: 1910명
사용할 변수(Feature) 개수: 862개


In [10]:
# 3. XGBoost 회귀 모델 생성 및 학습
model = xgb.XGBRegressor(
    n_estimators=100,     # 나무의 개수
    learning_rate=0.05,   # 학습률
    max_depth=6,          # 깊이
    random_state=42,
    n_jobs=-1             # 모든 CPU 코어 사용
)

model.fit(X, y)
print("XGBoost 모델 학습 완료! 🚀")

XGBoost 모델 학습 완료! 🚀


In [ ]:
# 4. SHAP 값 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 5. 중요도 순위(Rank) 만들기
# 각 변수의 SHAP 절대값 평균을 구해서 영향력 측정
shap_sum = np.abs(shap_values).mean(axis=0)

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'SHAP_Importance': shap_sum
})

# 중요도가 높은 순서대로 정렬
importance_df = importance_df.sort_values(by='SHAP_Importance', ascending=False)

print("\n📊 [SHAP Feature Importance Top 10] 📊")
print(importance_df.head(10))

# 6. 시각화 (요약 차트)
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X, plot_type="bar", show=False)
plt.title("Top Feature Importance by SHAP Value")
plt.show()

In [13]:
import pandas as pd
import numpy as np
import shap
from sklearn.ensemble import GradientBoostingRegressor

# 1. 데이터 준비 (이미 X, y가 로드되어 있다고 가정할게!)
# 오류가 났던 부분 해결: 빈 값(NaN)을 0으로 채워주기
X = X.fillna(0)

print(f"빈 값 처리 완료! (데이터 개수: {len(X)}개)")

# 2. 모델 학습
# (XGBoost 대신 sklearn의 GradientBoosting 사용 중)
model = GradientBoostingRegressor(
    n_estimators=100, 
    max_depth=4, 
    random_state=42
)
model.fit(X, y)
print("모델 학습 완료! 🚀")

# 3. SHAP 지수 뽑기 (전체 데이터 대상)
# TreeExplainer를 사용하여 SHAP 값 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 4. 순위 및 SHAP 중요도 정리
# 각 변수의 SHAP 절대값 평균으로 중요도 산출
shap_sum = np.abs(shap_values).mean(axis=0)

importance_df = pd.DataFrame({
    'Feature': X.columns,
    'SHAP_Importance': shap_sum
})

# 중요도가 높은 순서대로 정렬 및 순위 매기기
importance_df = importance_df.sort_values(by='SHAP_Importance', ascending=False).reset_index(drop=True)
importance_df['Rank'] = importance_df.index + 1

# 5. CSV 파일로 저장
importance_df.to_csv('VIP_SHAP_Importance.csv', index=False, encoding='utf-8-sig')

print("\n📊 [SHAP Feature Importance Top 10] 📊")
print(importance_df.head(10))
print("\n'VIP_SHAP_Importance.csv' 파일로 저장했어! 확인해봐! 😊")

빈 값 처리 완료! (데이터 개수: 1910개)
모델 학습 완료! 🚀

📊 [SHAP Feature Importance Top 10] 📊
            Feature  SHAP_Importance  Rank
0       Slope_Spend         4.299572     1
1     Slope_Balance         2.770525     2
2       Slope_Count         2.444372     3
3        잔액_일시불_B0M         0.007571     4
4   증감율_이용건수_일시불_분기         0.007512     5
5         납부_기타이용금액         0.006238     6
6           이용금액_교통         0.005632     7
7        정상입금원금_B0M         0.005334     8
8        승인거절건수_B0M         0.005210     9
9  할부금액_유이자_3M_R12M         0.005027    10

'VIP_SHAP_Importance.csv' 파일로 저장했어! 확인해봐! 😊
